In [3]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [6]:
# url 접속
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()
driver.get('https://www.drmartens.co.kr/')
time.sleep(2)  # 2초간 정지

In [157]:
# 남성 선택 
selector = "#HeaderMenu-남성"
driver.find_element(By.CSS_SELECTOR, selector).click()

In [7]:
# best 상품 클릭
selector = "#shopify-section-template--18223533686939__drmt_collection_header_BwmJLi > div > div.plp-top-menu-wrap > div > ul > li:nth-child(2)"
driver.find_element(By.CSS_SELECTOR, selector).click()
time.sleep(1)

In [ ]:
# step1. 상품 이름, url 크롤링
# step2. 첫번쨰 상품 url 들어가서 데이터 크롤링
# step3. 모든 상품 url 반복문으로 들어가서 데이터 크롤링
# step4. 시각화 및 인사이트 도출

# step1. 상품 이름, url 크롤링

### 남성 베스트 상품 및 url 수집

In [8]:
# 더보기 클릭
selector = ".buttonWrap"
driver.find_element(By.CSS_SELECTOR, selector).click()

In [10]:
# 상품 개수 파악
selector = "#product-grid > li"
product_ct = len(driver.find_elements(By.CSS_SELECTOR, selector))
product_ct

27

In [9]:
# 상품 개수 - 이미지로 크롤링했을떄
selector = ".card__inner.color-scheme-2.gradient.ratio"
product_cnt = len(driver.find_elements(By.CSS_SELECTOR, selector))
product_cnt

27

In [11]:
# 남자 베스트 첫번째 상품명
selector = "#product-grid > li:nth-child(1) > div > div > div.card__content"
title = driver.find_element(By.CSS_SELECTOR, selector).text
title.split('\n')[0]

'1461 스무스'

In [12]:
# 남자 베스트 상품 리스트
title_selector = "#product-grid > li"
title_cnt = len(driver.find_elements(By.CSS_SELECTOR, title_selector))

title_list = []
for i in range(1, title_cnt+1):
    selector = f"#product-grid > li:nth-child({i}) > div > div > div.card__content"
    tt = driver.find_element(By.CSS_SELECTOR, selector).text
    tt = tt.split('\n')[0]
    title_list.append(tt)

print(len(title_list))
title_list    

27


['1461 스무스',
 '1461 크레이지호스',
 '1461 쿼드',
 '1461 모노',
 '1461 나파 (무광)',
 '1461 벡스',
 '1461 벡스 모노',
 '리더',
 '1461 WS',
 '2976 모노',
 '브루클라인',
 '1461',
 '1460 스무스',
 '아드리안',
 '아드리안',
 '제이든 III',
 '1460 크레이지호스',
 '로웰',
 '1461 앰버서더',
 '로웰',
 '제이든',
 '아드리안',
 '로웰',
 '브루클라인',
 '브루클라인 로우',
 '산 뮬',
 '1460 백스']

In [13]:
#남자 베스트 첫번째 상품 url
selector = "#product-grid > li:nth-child(1) a"
url = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('href')
url

'https://www.drmartens.co.kr/products/boots-1461-black-smooth'

In [14]:
# 남자 베스트 상품 url 리스트
url_list = []
for i in range(1,28):
    selector = f"#product-grid > li:nth-child({i}) a"
    url = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('href')
    url_list.append(url)

print(len(url_list))
url_list

27


['https://www.drmartens.co.kr/products/boots-1461-black-smooth',
 'https://www.drmartens.co.kr/products/boots-1461-lace-shoe-gaucho-crazy-horse',
 'https://www.drmartens.co.kr/products/shoes-1461-quad-retro-25567001',
 'https://www.drmartens.co.kr/products/shoes-1461-mono-black-14345001',
 'https://www.drmartens.co.kr/products/1461-napa-mugwang',
 'https://www.drmartens.co.kr/products/shoes-1461-bex-black-smooth',
 'https://www.drmartens.co.kr/products/shoes-1461-bex-mono-black-41774001',
 'https://www.drmartens.co.kr/products/shoes-reeder-black-27104001',
 'https://www.drmartens.co.kr/products/shoes-1461-black-smooth',
 'https://www.drmartens.co.kr/products/boots-2976-mono-black-smooth',
 'https://www.drmartens.co.kr/products/shoes-brookline-41542001',
 'https://www.drmartens.co.kr/products/1461',
 'https://www.drmartens.co.kr/products/1460-2',
 'https://www.drmartens.co.kr/products/shoes-adrian',
 'https://www.drmartens.co.kr/products/adeurian',
 'https://www.drmartens.co.kr/products

In [15]:
# 남성 DataFrame 생성 
import pandas as pd
df_man = pd.DataFrame({"gender": ["남성"] * len(title_list), "title": title_list, "url": url_list})
df_man

,gender,title,url
0,남성,1461 스무스,https://www.drmartens.co.kr/products/boots-146...
1,남성,1461 크레이지호스,https://www.drmartens.co.kr/products/boots-146...
2,남성,1461 쿼드,https://www.drmartens.co.kr/products/shoes-146...
3,남성,1461 모노,https://www.drmartens.co.kr/products/shoes-146...
4,남성,1461 나파 (무광),https://www.drmartens.co.kr/products/1461-napa...
5,남성,1461 벡스,https://www.drmartens.co.kr/products/shoes-146...
6,남성,1461 벡스 모노,https://www.drmartens.co.kr/products/shoes-146...
7,남성,리더,https://www.drmartens.co.kr/products/shoes-ree...
8,남성,1461 WS,https://www.drmartens.co.kr/products/shoes-146...
9,남성,2976 모노,https://www.drmartens.co.kr/products/boots-297...


### 여성 best 상품 및 url 수집

In [17]:
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저

driver.get('https://www.drmartens.co.kr/collections/all-men')
time.sleep(2)  # 2초간 정지

In [18]:
#여성 선택
selector = "#HeaderMenu-여성"
driver.find_element(By.CSS_SELECTOR, selector).click()

In [19]:
# best 상품 클릭
selector = "#shopify-section-template--18223532966043__drmt_collection_header_BwmJLi > div > div.plp-top-menu-wrap > div > ul > li:nth-child(2)"
driver.find_element(By.CSS_SELECTOR, selector).click()
time.sleep(1)

In [21]:
# 더보기 클릭 
selector = ".buttonWrap"
driver.find_element(By.CSS_SELECTOR, selector).click()

In [22]:
# 상품 개수 
selector = "#product-grid > li"
product_w_ct = len(driver.find_elements(By.CSS_SELECTOR, selector))
product_w_ct

34

In [23]:
# 여자 베스트 첫번째 상품명
selector = "#product-grid > li:nth-child(1) > div > div > div.card__content"
w_title = driver.find_element(By.CSS_SELECTOR, selector).text
w_title.split('\n')[0]

'1461 스무스'

In [24]:
# 여자 베스트 상품 리스트
w_title_selector = "#product-grid > li"
title_cnt = len(driver.find_elements(By.CSS_SELECTOR, w_title_selector))

w_title_list = []
for i in range(1, title_cnt+1):
    selector = f"#product-grid > li:nth-child({i}) > div > div > div.card__content"
    w_title = driver.find_element(By.CSS_SELECTOR, selector).text
    w_title = w_title.split('\n')[0]
    w_title_list.append(w_title)

print(len(w_title_list))
w_title_list

34


['1461 스무스',
 '1461 크레이지호스',
 '1461 쿼드',
 '1461 모노',
 '1461 나파 (무광)',
 '1461 벡스',
 '1461 벡스 모노',
 '리더',
 '1461 WS',
 '2976 모노',
 '브루클라인',
 '1461',
 '1460 스무스',
 '아드리안',
 '아드리안',
 '제이든 III',
 '1460 크레이지호스',
 '로웰',
 '8065 메리제인',
 '1461 앰버서더',
 '1B60',
 '로웰',
 '제이든',
 '아드리안',
 '로웰',
 '브루클라인',
 '브루클라인 로우',
 '1B60 벡스',
 '산 뮬',
 '폴리',
 '아마야 하이',
 '1b99',
 '1460 백스',
 '메이볼 메리제인']

In [25]:
# 여자 베스트 상품 url
selector = "#product-grid > li:nth-child(1) a"
w_url = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('href')
w_url

'https://www.drmartens.co.kr/products/boots-1461-black-smooth'

In [26]:
w_url_list = []
for i in range(1,35):
    selector = f"#product-grid > li:nth-child({i}) a"
    w_url = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('href')
    w_url_list.append(w_url)

print(len(w_url_list))
w_url_list

34


['https://www.drmartens.co.kr/products/boots-1461-black-smooth',
 'https://www.drmartens.co.kr/products/boots-1461-lace-shoe-gaucho-crazy-horse',
 'https://www.drmartens.co.kr/products/shoes-1461-quad-retro-25567001',
 'https://www.drmartens.co.kr/products/shoes-1461-mono-black-14345001',
 'https://www.drmartens.co.kr/products/1461-napa-mugwang',
 'https://www.drmartens.co.kr/products/shoes-1461-bex-black-smooth',
 'https://www.drmartens.co.kr/products/shoes-1461-bex-mono-black-41774001',
 'https://www.drmartens.co.kr/products/shoes-reeder-black-27104001',
 'https://www.drmartens.co.kr/products/shoes-1461-black-smooth',
 'https://www.drmartens.co.kr/products/boots-2976-mono-black-smooth',
 'https://www.drmartens.co.kr/products/shoes-brookline-41542001',
 'https://www.drmartens.co.kr/products/1461',
 'https://www.drmartens.co.kr/products/1460-2',
 'https://www.drmartens.co.kr/products/shoes-adrian',
 'https://www.drmartens.co.kr/products/adeurian',
 'https://www.drmartens.co.kr/products

In [27]:
# 여성 상품 DataFrame 생성
df_women = pd.DataFrame({"gender": ["여성"] * len(w_title_list),"title": w_title_list, "url": w_url_list})
df_women

,gender,title,url
0,여성,1461 스무스,https://www.drmartens.co.kr/products/boots-146...
1,여성,1461 크레이지호스,https://www.drmartens.co.kr/products/boots-146...
2,여성,1461 쿼드,https://www.drmartens.co.kr/products/shoes-146...
3,여성,1461 모노,https://www.drmartens.co.kr/products/shoes-146...
4,여성,1461 나파 (무광),https://www.drmartens.co.kr/products/1461-napa...
5,여성,1461 벡스,https://www.drmartens.co.kr/products/shoes-146...
6,여성,1461 벡스 모노,https://www.drmartens.co.kr/products/shoes-146...
7,여성,리더,https://www.drmartens.co.kr/products/shoes-ree...
8,여성,1461 WS,https://www.drmartens.co.kr/products/shoes-146...
9,여성,2976 모노,https://www.drmartens.co.kr/products/boots-297...


In [29]:
#남성 여성 DataFrame 합치기
df_all = pd.concat([df_man, df_women], ignore_index=True)
df_all

,gender,title,url
0,남성,1461 스무스,https://www.drmartens.co.kr/products/boots-146...
1,남성,1461 크레이지호스,https://www.drmartens.co.kr/products/boots-146...
2,남성,1461 쿼드,https://www.drmartens.co.kr/products/shoes-146...
3,남성,1461 모노,https://www.drmartens.co.kr/products/shoes-146...
4,남성,1461 나파 (무광),https://www.drmartens.co.kr/products/1461-napa...
...,...,...,...
56,여성,폴리,https://www.drmartens.co.kr/products/shoes-pol...
57,여성,아마야 하이,https://www.drmartens.co.kr/products/shoes-ama...
58,여성,1b99,https://www.drmartens.co.kr/products/1b99
59,여성,1460 백스,https://www.drmartens.co.kr/products/boots-146...


In [30]:
# gender가 '여성'인 데이터만 보기
# 남성 : 0 ~ 26 / 여성은 27 ~ 60
df_all[df_all['gender'] == '여성']

,gender,title,url
27,여성,1461 스무스,https://www.drmartens.co.kr/products/boots-146...
28,여성,1461 크레이지호스,https://www.drmartens.co.kr/products/boots-146...
29,여성,1461 쿼드,https://www.drmartens.co.kr/products/shoes-146...
30,여성,1461 모노,https://www.drmartens.co.kr/products/shoes-146...
31,여성,1461 나파 (무광),https://www.drmartens.co.kr/products/1461-napa...
32,여성,1461 벡스,https://www.drmartens.co.kr/products/shoes-146...
33,여성,1461 벡스 모노,https://www.drmartens.co.kr/products/shoes-146...
34,여성,리더,https://www.drmartens.co.kr/products/shoes-ree...
35,여성,1461 WS,https://www.drmartens.co.kr/products/shoes-146...
36,여성,2976 모노,https://www.drmartens.co.kr/products/boots-297...


# step2 각 url 정보 수집 (상품명, 가격, 평점, 리뷰수)

In [31]:
# 첫번째 상품 url 접속
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저

i = 0
url = df_all['url'][i]
driver.get(url)
time.sleep(2)

In [32]:
# 제품명 수집
selector = ".product__title"
title = driver.find_element(By.CSS_SELECTOR, selector).text
title

'1461 스무스'

In [33]:
# 색상 수집
selector = ".product-option"
color = driver.find_element(By.CSS_SELECTOR, selector).text
color = color.split('\n')[1].split('/')[0]
color

'BLK '

In [34]:
# 가격 수집
selector = ".product__buy_price"
price = driver.find_element(By.CSS_SELECTOR, selector).text
price = int(price.split('\n')[1].replace('₩','').replace(',', ''))
price

229000

In [35]:
#리뷰수
selector = ".crema-product-reviews-score-link"
review_cnt = driver.find_element(By.CSS_SELECTOR, selector).text
review_cnt = int(review_cnt.split(' ')[0])
review_cnt

266

In [36]:
# iframe 접근
driver.switch_to.frame("crema-product-reviews-1")

In [37]:
#평점
selector = "#app > div > div > div.AppAjaxContent > div > div > div > div > div > div.AppHeaderV2 > div.AppAjaxContent > div > div > div > div.AppHeaderV2ScoreSection__product-score.AppHeaderV2ProductScore > div.AppHeaderV2ProductScore__score"
rating = float(driver.find_element(By.CSS_SELECTOR, selector).text)
rating

4.9

In [38]:
# 리뷰 내용 수집
selector = "div.BodyPc__review"
driver.find_element(By.CSS_SELECTOR, selector).text

'상품 옵션\n260\n남자친구 선물이었는데, 너무 좋아하네요!\n신고 및 차단\n도움돼요\n도움안돼요\n댓글\n0\n임****\n작성자 등급회원\n2026. 1. 25.'

In [39]:
# 리뷰 내용
selector = "div.BodyPc__review"
reviews = driver.find_elements(By.CSS_SELECTOR, selector)
reviews[0].text.split('\n')[2]

'남자친구 선물이었는데, 너무 좋아하네요!'

In [40]:
# 1p 리뷰 내용 전체 수집
review_str = ""
for r in reviews:
    review = r.text.split('\n')[2]
    review_str = review_str + review

review_str

'남자친구 선물이었는데, 너무 좋아하네요!길들이는데 좀 걸리지만 디자인은 최고입니다.너무 좋아요!!무척 예뻐요. 발도 그리 아프지 않았고 좋아요. 편하고 이뻐요. 왜 아프다고 하는지 모르겠어요. 생각보다 발 안아프고 너무 이쁘게 휴가기간에 잘 신었습니다1460도 구매 해야겠어요 ㅠㅠ'

In [41]:
# 수집정보 딕셔너리로 저장
target_info = {}
target_info['product'] = title
target_info['color'] = color
target_info['price'] = price
target_info['review_cnt'] = review_cnt
target_info['rating'] = rating
target_info['review'] = review_str

In [42]:
df_all.head(3)

,gender,title,url
0,남성,1461 스무스,https://www.drmartens.co.kr/products/boots-146...
1,남성,1461 크레이지호스,https://www.drmartens.co.kr/products/boots-146...
2,남성,1461 쿼드,https://www.drmartens.co.kr/products/shoes-146...


In [43]:
url = df_all['url']
url

0     https://www.drmartens.co.kr/products/boots-146...
1     https://www.drmartens.co.kr/products/boots-146...
2     https://www.drmartens.co.kr/products/shoes-146...
3     https://www.drmartens.co.kr/products/shoes-146...
4     https://www.drmartens.co.kr/products/1461-napa...
                            ...                        
56    https://www.drmartens.co.kr/products/shoes-pol...
57    https://www.drmartens.co.kr/products/shoes-ama...
58            https://www.drmartens.co.kr/products/1b99
59    https://www.drmartens.co.kr/products/boots-146...
60    https://www.drmartens.co.kr/products/shoes-may...
Name: url, Length: 61, dtype: object

In [44]:
# 1번째 url 정보 잘 수집되는지 확인
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저

i = 0
driver.get(url[i])
time.sleep(1)

target_info = {}

# 제품명 수집
selector = ".product__title"
title = driver.find_element(By.CSS_SELECTOR, selector).text

# 색상 수집
selector = ".product-option"
color = driver.find_element(By.CSS_SELECTOR, selector).text
color = color.split('\n')[1].split('/')[0]

# 가격 수집
selector = ".product__buy_price"
price = driver.find_element(By.CSS_SELECTOR, selector).text
price = int(price.split('\n')[1].replace('₩','').replace(',', ''))

#리뷰수
selector = ".crema-product-reviews-score-link"
review_cnt = driver.find_element(By.CSS_SELECTOR, selector).text
review_cnt = int(review_cnt.split(' ')[0])

# iframe 접근
driver.switch_to.frame("crema-product-reviews-1")

#평점
selector = "#app > div > div > div.AppAjaxContent > div > div > div > div > div > div.AppHeaderV2 > div.AppAjaxContent > div > div > div > div.AppHeaderV2ScoreSection__product-score.AppHeaderV2ProductScore > div.AppHeaderV2ProductScore__score"
rating = float(driver.find_element(By.CSS_SELECTOR, selector).text)

# 리뷰 수집
selector = "div.BodyPc__review"
reviews = driver.find_elements(By.CSS_SELECTOR, selector)
reviews[0].text.split('\n')[2]
review_str = ""
for r in reviews:
    review = r.text.split('\n')[2]
    review_str = review_str + review

# 수집정보 딕셔너리로 저장
target_info = {}
target_info['product'] = title
target_info['color'] = color
target_info['price'] = price
target_info['review_cnt'] = review_cnt
target_info['rating'] = rating
target_info['review'] = review_str

target_info

{'product': '1461 스무스',
 'color': 'BLK ',
 'price': 229000,
 'review_cnt': 266,
 'rating': 4.9,
 'review': '남자친구 선물이었는데, 너무 좋아하네요!길들이는데 좀 걸리지만 디자인은 최고입니다.너무 좋아요!!무척 예뻐요. 발도 그리 아프지 않았고 좋아요. 편하고 이뻐요. 왜 아프다고 하는지 모르겠어요. 생각보다 발 안아프고 너무 이쁘게 휴가기간에 잘 신었습니다1460도 구매 해야겠어요 ㅠㅠ'}

# step3 for문으로 전체 수집

In [45]:
# range 범위 
shoes_cnt = len(df_all['url'])
shoes_cnt

61

In [46]:
url = df_all['url']
url

0     https://www.drmartens.co.kr/products/boots-146...
1     https://www.drmartens.co.kr/products/boots-146...
2     https://www.drmartens.co.kr/products/shoes-146...
3     https://www.drmartens.co.kr/products/shoes-146...
4     https://www.drmartens.co.kr/products/1461-napa...
                            ...                        
56    https://www.drmartens.co.kr/products/shoes-pol...
57    https://www.drmartens.co.kr/products/shoes-ama...
58            https://www.drmartens.co.kr/products/1b99
59    https://www.drmartens.co.kr/products/boots-146...
60    https://www.drmartens.co.kr/products/shoes-may...
Name: url, Length: 61, dtype: object

In [48]:
#for 문으로 전체 크롤링
chromedriver_autoinstaller.install(cwd=True)

total_dict = {}

for i in range(len(url)):  # url = df_all['url']
    try:
        driver = webdriver.Chrome()
        driver.get(url[i])
        time.sleep(1)
        
        target_info = {}
        
        # 제품명 수집
        selector = ".product__title"
        try:
            title = driver.find_element(By.CSS_SELECTOR, selector).text
        except:
            title = '알수없음'
        
        # 색상 수집
        selector = ".product-option"
        try:
            color = driver.find_element(By.CSS_SELECTOR, selector).text
            color = color.split('\n')[1].split('/')[0]
        except: color = '알수없음'
        
        # 가격 수집
        selector = ".product__buy_price"
        try:
            price = driver.find_element(By.CSS_SELECTOR, selector).text
            price = int(price.split('\n')[1].replace('₩','').replace(',', ''))
        except:
            price = 0

        #리뷰수
        selector = ".crema-product-reviews-score-link"
        try:
            review_cnt = driver.find_element(By.CSS_SELECTOR, selector).text
            review_cnt = int(review_cnt.split(' ')[0])
        except:
            review_cnt = 0
        
        # iframe 접근
        driver.switch_to.frame("crema-product-reviews-1")
        
        #평점
        selector = "#app > div > div > div.AppAjaxContent > div > div > div > div > div > div.AppHeaderV2 > div.AppAjaxContent > div > div > div > div.AppHeaderV2ScoreSection__product-score.AppHeaderV2ProductScore > div.AppHeaderV2ProductScore__score"
        try:
            rating = float(driver.find_element(By.CSS_SELECTOR, selector).text)
        except:
            rating = 0
        
        # 리뷰 수집
        selector = "div.BodyPc__review"
        reviews = driver.find_elements(By.CSS_SELECTOR, selector)
        try:
            reviews[0].text.split('\n')[2]
            review_str = ""
            for r in reviews:
                try:
                    review = r.text.split('\n')[2]
                    review_str = review_str + review
                except:
                    continue
        except:
            review_str = "리뷰없음"

        # iframe 접근 해제
        driver.switch_to.default_content()

        # 수집정보 딕셔너리로 저장
        target_info = {}
        target_info['product'] = title
        target_info['color'] = color
        target_info['price'] = price
        target_info['review_cnt'] = review_cnt
        target_info['rating'] = rating
        target_info['review'] = review_str
              
        print(target_info)
        total_dict[i] = target_info
        print(i, "번째 데이터가 수집되었습니다.")
        time.sleep(2)
        driver.close()
    except:
        print(i, "번째 데이터는 건너뜁니다.")
        driver.close()
        continue

{'product': '1461 스무스', 'color': 'BLK ', 'price': 229000, 'review_cnt': 266, 'rating': 4.9, 'review': '남자친구 선물이었는데, 너무 좋아하네요!길들이는데 좀 걸리지만 디자인은 최고입니다.너무 좋아요!!무척 예뻐요. 발도 그리 아프지 않았고 좋아요. 편하고 이뻐요. 왜 아프다고 하는지 모르겠어요. 생각보다 발 안아프고 너무 이쁘게 휴가기간에 잘 신었습니다1460도 구매 해야겠어요 ㅠㅠ'}
0 번째 데이터가 수집되었습니다.
{'product': '1461 크레이지호스', 'color': 'DRB ', 'price': 229000, 'review_cnt': 69, 'rating': 4.6, 'review': '정말 오래 기다렸습니다... 오프라인 매장도 다가보고 헛걸음했는데 갑자기 온라인으로 재고들어왔다구 해서 바로 주문했어요!! 기다린만큼 너무 이쁘고, 매일 신고있습니다!!재입고 될때까지 기다렸다가 구매했는데편하고 좋아요 오리지날 닥터마틴보다 편해요 색도 예뻐요배송도 빠르고 디자인도 마음에 들어서 좋습니다!!구두인데 착용감 너무 좋고 너무 이뻐요 '}
1 번째 데이터가 수집되었습니다.
{'product': '1461 쿼드', 'color': 'BLK ', 'price': 269000, 'review_cnt': 42, 'rating': 4.8, 'review': '딸아이 신고싶다고 해서 사준건데 사이즈때문에 교환은 했지만 너무 좋아해요. 나갈때마다 신어요. 좋아하는 모습을 보니 저도 좋습니다반품시킨지 한 달이 지났는데, 반품이 되는지 안되는지 안내라도 좀 해주시죠.벽돌같은 무게감에 깜짝 놀라시겠지만(오른쪽 구두) 1461 쿼드 너무 예뻐요!!! 구두 하나도 없어서 사고 싶었는데 닥터마틴이 제일 예쁜 것 같아서 샀어요 비싸지만 이거 평생 신을래요 굽 있는 것도 좋어용미쳤습니다, 이게 근본이죠'}
2 번째 데이터가 수집되었습니다.
{'product': '1461 모노', 'color': '

In [52]:
# 상세제품 데이터프레임
result_df = pd.DataFrame.from_dict(total_dict, 'index')
print(result_df.shape)
result_df

(61, 6)


,product,color,price,review_cnt,rating,review
0,1461 스무스,BLK,229000,266,4.9,"남자친구 선물이었는데, 너무 좋아하네요!길들이는데 좀 걸리지만 디자인은 최고입니다...."
1,1461 크레이지호스,DRB,229000,69,4.6,정말 오래 기다렸습니다... 오프라인 매장도 다가보고 헛걸음했는데 갑자기 온라인으로...
2,1461 쿼드,BLK,269000,42,4.8,딸아이 신고싶다고 해서 사준건데 사이즈때문에 교환은 했지만 너무 좋아해요. 나갈때마...
3,1461 모노,BLK,229000,417,4.8,닥터마틴 길들이는데 힘들다고해소 걱정하고 사서 신었는데 소문과는 다르게 편하게 잘 ...
4,1461 나파 (무광),BLK,229000,116,4.8,물건 좋네요 수고하세요 착화감이 좋네요 이쁘고 좋아요타사 제품 사용하다가 유투브로...
...,...,...,...,...,...,...
56,폴리,BLK,168000,0,0.0,리뷰없음
57,아마야 하이,BLK,320000,1,5.0,지퍼라 편하고 너무이쁨 다리도 얇아보입니다
58,1b99,BLK,309000,59,4.9,운동화 245신는 발볼넓임 제품은 240 그렇게 아프지도 않고 이부다신고 및 차단땀...
59,1460 백스,BLK,280000,25,5.0,청바지랑 잘 어울려요.신고 및 차단도움돼요빠른 답변과 교환으로 편하게 물건을 잘 받...


In [57]:
# 남성 : 0 ~ 26 / 여성은 27 ~ 60
# 1. gender 컬럼 생성
result_df['gender'] = ['남자' if i <= 26 else '여자' for i in result_df.index]

# 2. gender 컬럼을 맨 앞으로 이동
cols = ['gender'] + [col for col in result_df.columns if col != 'gender']

result_df = result_df[cols]
result_df

,gender,product,color,price,review_cnt,rating,review
0,남자,1461 스무스,BLK,229000,266,4.9,"남자친구 선물이었는데, 너무 좋아하네요!길들이는데 좀 걸리지만 디자인은 최고입니다...."
1,남자,1461 크레이지호스,DRB,229000,69,4.6,정말 오래 기다렸습니다... 오프라인 매장도 다가보고 헛걸음했는데 갑자기 온라인으로...
2,남자,1461 쿼드,BLK,269000,42,4.8,딸아이 신고싶다고 해서 사준건데 사이즈때문에 교환은 했지만 너무 좋아해요. 나갈때마...
3,남자,1461 모노,BLK,229000,417,4.8,닥터마틴 길들이는데 힘들다고해소 걱정하고 사서 신었는데 소문과는 다르게 편하게 잘 ...
4,남자,1461 나파 (무광),BLK,229000,116,4.8,물건 좋네요 수고하세요 착화감이 좋네요 이쁘고 좋아요타사 제품 사용하다가 유투브로...
...,...,...,...,...,...,...,...
56,여자,폴리,BLK,168000,0,0.0,리뷰없음
57,여자,아마야 하이,BLK,320000,1,5.0,지퍼라 편하고 너무이쁨 다리도 얇아보입니다
58,여자,1b99,BLK,309000,59,4.9,운동화 245신는 발볼넓임 제품은 240 그렇게 아프지도 않고 이부다신고 및 차단땀...
59,여자,1460 백스,BLK,280000,25,5.0,청바지랑 잘 어울려요.신고 및 차단도움돼요빠른 답변과 교환으로 편하게 물건을 잘 받...


In [58]:
# CSV 파일로 저장
result_df.to_csv('닥터마틴_best_df.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 시각화 시작